In [1]:
#variables setup for fineuting
import pickle 
import pandas as pd
import os
import json
import torch
import Libs.DataSynthesisLib as DataSynthesisLib
import re
import sys
import random
from sys import exit
from multiprocess import set_start_method
from transformers import AutoModelForCausalLM,AutoTokenizer
from peft import PeftModel
from datasets import Dataset
from peft import PeftModel
import torch.multiprocessing  as mp

os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"
os.environ["WANDB_DISABLED"] = "true"
GPU_ID="0"
os.environ["CUDA_VISIBLE_DEVICES"]=GPU_ID
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

Config=DataSynthesisLib.GetConfig('Config/Config.py')
Vendor="unsloth"
BaseModel="Meta-Llama-3.1-8B-Instruct"


QuantModel=BaseModel+""
UnQuantModel=BaseModel

#Output model path
TargetDir="/tmp/"
TargetDirOutModel="/tmp/"

UnQuantModelVendor=Vendor+"/"+UnQuantModel

ModelFT=Vendor+"/"+UnQuantModel
FilenameShort='AllComments_Clean_Train'
Filename='Data/AllComments_Clean_Train'

with open(Filename + ".json", "r") as f:
    TrainData=json.load(f)




/home/cdsw/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-21 21:00:05,672	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/cdsw/.local/lib/python3.10/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


In [2]:
#Finetuning parameters
Data_Size=5000
#########CHange per model
AssistantToken="<|start_header_id|>assistant"
TrimBeg=1
Filename2=Filename+"_Datasize_"+str(Data_Size) +'_'+BaseModel
ConfigS=Config["ConfigSolution"].copy()
ConfigS["lora_r"]=128
ConfigS["target_modules"]=['q_proj','k_proj','v_proj','o_proj']
ConfigS["target_modules"]="all-linear"
ConfigS["lora_alpha"]=64
ConfigS["lora_dropout"]=0.05

ConfigS["gpu-id"]=GPU_ID

ConfigS["max_model_len"]="8000"
ConfigS["max_num_batched_tokens"]="8000"
ConfigS["llm-path"]=ModelFT
ConfigS["tokenizer-path"]=ModelFT
ConfigS["llm-path"]=ModelFT
ConfigS["tokenizer-path"]=ModelFT

ConfigS["tokenizer-file-gguf"]=""
ConfigS["tensor_parallel_size"]="1"
ConfigS["GenerationMode"]="InferenceProcessBlocked"
ConfigS["use-grammar"]="False"
ConfigS["BlockSize"]="400"
ConfigS["FT-num_train_epochs"]="1"

ConfigS["AssistantPrompt"]="Sure, here is the python solution:\n```python"
ConfigS["GenerateChatTemplate"]="GenerateChatWithSystemWithAssistant"
ConfigS["UseOutlines"]="False"
ConfigS["AssistantPromptElementsRemoval"]="2"
ConfigS["FT-SaveModel"]=TargetDir+Filename2
ConfigS["FT-SaveModel"]=TargetDir+"all-linear"+FilenameShort+"r_"+str(ConfigS["lora_r"])+'_'+'a_'+str(ConfigS["lora_alpha"])+'_'+'d_'+str(ConfigS["lora_dropout"])

os.environ["CUDA_VISIBLE_DEVICES"] = ConfigS["gpu-id"]
os.environ["NCCL_SHM_DISABLE"]="1"



In [3]:
#Training data examples
print(TrainData[0]['Prompt'])
print('--------------------')
print(TrainData[1]['Prompt'])

You are given a Cloudera support team comment, 11 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the Cloudera comment:
    
Hi Support Team,

Since the latest update to version 3.6.5, we're experiencing frequent data loading errors in the mobile app. The app shows 'Failed to Load Data' messages intermittently when accessing any dashboard, making it unusable for our field teams. This appears to be happening across both iOS and Android devices.

We've already:
- Updated to the latest version
- Cleared app cache
- Verified network connectivity
- Tested on multiple devices

This is affecting our entire field operations team of 100+ people who need access to real-time analytics.

Please investigate this critical issue.

Best regards,
Andrew
Field Operations Director

Here are the 11 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Score the severity of the issue ba

In [4]:
#Convert input training data to chat completion

tokenizer=AutoTokenizer.from_pretrained(ModelFT)

# Helper function to format the dataset
def chatml_format(Data,tokenizer,Data_Size):
  Out=[]
  OutFormat2={
        "text": []
  }
  for i in range(0,min(len(Data['Questions']),Data_Size)):
    #if Data['UnitTestsPass'][i]==True:

      message = [{"role": "system",    "content": Data['system']},
                 {"role": "user",      "content": Data['Questions'][i]},
                 {"role": "assistant", "content": Data["SolutionsCleanedPython"][i]}]
      system_user_completion=tokenizer.apply_chat_template(message, tokenize=False,add_generation_prompt=False)

      OutFormat2["text"].append(system_user_completion)
      d={}
      d['text']=system_user_completion
      Out.append(d)
  return (OutFormat2,Out)

MyDatasetTrain=TrainData.copy()

MyDatasetTrain={}
MyDatasetTrain["system"]='You are a helpful assistant.'
MyDatasetTrain["Questions"]=[i["Prompt"] for i in TrainData]
MyDatasetTrain["SolutionsCleanedPython"]=[i["Completion"] for i in TrainData]


(MyDatasetTrainSFT,MyDatasetTrainSFTJson)=chatml_format(MyDatasetTrain,tokenizer,Data_Size)


In [5]:
#Split data into dev/training dataset (90% training)

ds_len=len(MyDatasetTrainSFT['text'])
df_test = pd.DataFrame.from_dict(MyDatasetTrainSFT)[9*ds_len//10:]
df_train = pd.DataFrame.from_dict(MyDatasetTrainSFT)[:9*ds_len//10]
dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)


In [6]:
dataset_train

Dataset({
    features: ['text'],
    num_rows: 4500
})

In [7]:
#Do finetuning

os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
def SFTTrain(ConfigS,Input):
  
  import os
  os.environ["CUDA_VISIBLE_DEVICES"]=ConfigS["gpu-id"]
  from peft import LoraModel, LoraConfig
  import peft
  import torch
  from trl import SFTConfig, SFTTrainer
  from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
  from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, DataCollatorForSeq2Seq #,infer_auto_device_map
  from peft import PeftModel, PeftConfig, AutoPeftModelForCausalLM
  from peft import PeftModel, PeftConfig, AutoPeftModelForCausalLM
  device_map={'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 0, 'model.layers.11': 0, 'model.layers.12': 0, 'model.layers.13': 0, 'model.layers.14': 0, 'model.layers.15': 0, 'model.layers.16': 0, 'model.layers.17': 0, 'model.layers.18': 0, 'model.layers.19': 0, 'model.layers.20': 0, 'model.layers.21': 0, 'model.layers.22': 0, 'model.layers.23': 0, 'model.layers.24': 0, 'model.layers.25': 0, 'model.layers.26': 0, 'model.layers.27': 0, 'model.layers.28': 0, 'model.layers.29': 0, 'model.layers.30': 0, 'model.layers.31': 0, 'model.layers.32': 0, 'model.layers.33': 0, 'model.layers.34': 0, 'model.layers.35': 0, 'model.layers.36': 0, 'model.layers.37': 0, 'model.layers.38': 0, 'model.layers.39': 0, 'model.layers.40': 0, 'model.layers.41': 0, 'model.layers.42': 0, 'model.layers.43': 0, 'model.layers.44': 0, 'model.layers.45': 1, 'model.layers.46': 1, 'model.layers.47': 1, 'model.layers.48': 1, 'model.layers.49': 1, 'model.layers.50': 1, 'model.layers.51': 1, 'model.layers.52': 1, 'model.layers.53': 1, 'model.layers.54': 1, 'model.layers.55': 1, 'model.layers.56': 1, 'model.layers.57': 1, 'model.layers.58': 1, 'model.layers.59': 1, 'model.layers.60': 1, 'model.layers.61': 1, 'model.layers.62': 1, 'model.layers.63': 1, 'model.norm': 1, 'model.rotary_emb': 0, 'lm_head': 1}
  dataset_train=Input["dataset_train"]
  dataset_test=Input["dataset_test"]




  model=AutoModelForCausalLM.from_pretrained(ConfigS["llm-path"], device_map="cuda",torch_dtype=torch.float16)
  
            
  tokenizer = AutoTokenizer. from_pretrained(ConfigS["llm-path"])

  model.gradient_checkpointing_enable()



  peft_config = LoraConfig(
    r=ConfigS["lora_r"],
    lora_alpha=ConfigS["lora_alpha"],
    target_modules=ConfigS["target_modules"],
    lora_dropout=ConfigS["lora_dropout"],
    bias="none",
    task_type="CAUSAL_LM",
  )
  model.enable_input_require_grads() 
  model = peft.get_peft_model(model, peft_config)
  optimizer=torch.optim.AdamW(model.parameters(),float(ConfigS["FT-learning_rate"]))
  training_args = SFTConfig(   
    max_seq_length=4096,
    output_dir=TargetDir+ConfigS["FT-output-dir"],
    do_train=bool(ConfigS["FT-do_train"]),
    do_eval=bool(ConfigS["FT-do_eval"]),
    per_device_train_batch_size=int(ConfigS["FT-per_device_train_batch_size"]),
    per_device_eval_batch_size=int(ConfigS["FT-per_device_eval_batch_size"]),
    learning_rate=float(ConfigS["FT-learning_rate"]),
    num_train_epochs=int(ConfigS["FT-num_train_epochs"]),
    logging_dir=ConfigS["FT-logging_dir"],
    logging_steps=int(ConfigS["FT-logging_steps"]),
    save_steps=int(ConfigS["FT-save_steps"]),
    optim=ConfigS["FT-optim"],

    dataset_text_field='text',

    gradient_accumulation_steps=int(ConfigS["gradient_accumulation_steps"]),
    eval_steps=10,
    fp16=True
  )

  
  trainer = SFTTrainer(
    model,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    args=training_args,
    tokenizer=tokenizer,
    optimizers=(optimizer,None)
  )
  res=trainer.train()

  trainer.save_model(ConfigS["FT-SaveModel"])
    
  return res

Input={}
Input["dataset_train"]=dataset_train
Input["dataset_test"]=dataset_test
#Launch finetuning
p=mp.Process(target=SFTTrain,args=(ConfigS,Input))
p.start()
p.join()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:04<00:00,  1.22s/it]
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_8011/35425205.py:71: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
Map: 100%|███████████████████████████| 500/500 [00:00<00:00, 3480.76 examples/s]
/home/cdsw/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overf

Step,Training Loss
10,1.412000
20,0.640400
30,0.344200
40,0.331600
50,0.295500
60,0.276200
70,0.255500
80,0.273600
90,0.262800
100,0.265900


/home/cdsw/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/cdsw/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/cdsw/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/cdsw/.l

TrainOutput(global_step=562, training_loss=0.2668869098734601, metrics={'train_runtime': 1561.2864, 'train_samples_per_second': 2.882, 'train_steps_per_second': 0.36, 'total_flos': 1.524387104004096e+17, 'train_loss': 0.2668869098734601, 'epoch': 0.9991111111111111})

In [8]:
TargetModel=TargetDirOutModel+'merged_'+FilenameShort+"r_"+str(ConfigS["lora_r"])+'_'+'a_'+str(ConfigS["lora_alpha"])+'_'+'d_'+str(ConfigS["lora_dropout"])
print(ModelFT)
print(TargetModel)


unsloth/Meta-Llama-3.1-8B-Instruct
/tmp/merged_AllComments_Clean_Trainr_128_a_64_d_0.05


In [1]:
#Merge model and adapters
def MergeAndSave(ModelFT, TargetModel):
  tokenizer=AutoTokenizer.from_pretrained(ModelFT)

  base_model = AutoModelForCausalLM.from_pretrained(ModelFT,torch_dtype=torch.float16)

  peft_model_id = ConfigS["FT-SaveModel"]
  model = PeftModel.from_pretrained(base_model, peft_model_id)

  merged_model = model.merge_and_unload()

  merged_model.save_pretrained(TargetModel)
  tokenizer.save_pretrained(TargetModel)

p=mp.Process(target=MergeAndSave,args=(ModelFT, TargetModel))
p.start()
p.join()


NameError: name 'AutoTokenizer' is not defined